In [0]:
!pip install lyricsgenius
!pip install -U pip
!pip install -U dill
!pip install -U nltk==3.5b1

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.0.2)
Requirement already up-to-date: dill in /usr/local/lib/python3.6/dist-packages (0.3.1.1)
Requirement already up-to-date: nltk==3.5b1 in /usr/local/lib/python3.6/dist-packages (3.5b1)


In [0]:
import lyricsgenius
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk import tokenize
from nltk.tokenize import LineTokenizer
from nltk.tokenize.treebank import TreebankWordTokenizer
from sklearn.model_selection import train_test_split
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm import MLE
from nltk.lm import Laplace
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
genius = lyricsgenius.Genius("2-vcV8P3rF4axVmDxH8NW9NB8j3tlT_M69vdKD60To_V7tZB7o8BWfm5uUETQ3Cg")
genius.remove_section_headers = True #We don't need headers
genius.skip_non_songs = False 
genius.excluded_terms = ["(Remix)", "(Live)"] #Remixes and live records are usually the same as original songs, so we do not need to double lyrics

In [0]:
#Let's start with 50 songs of Oxxxymiron. We can find total of 88 songs, but I've decided to start with 50, because it takes a lot of time to generate text with a big sample
artist = genius.search_artist("Oxxxymiron", max_songs=50, sort="title")

Searching for songs by Oxxxymiron...

Song 1: "16 Bars Acapella"
Song 2: "Afterparty (Demo)"
Song 3: "AI Ogon"
Song 4: "Amplify and simplify (Freestyle)"
Song 5: "CCTV"
Song 6: "Darkside"
Song 7: "Freestyle #1"
Song 8: "Ganz Promo Tune"
Song 9: "Hangover"
Song 10: "HPL"
Song 11: "IMPERIVM"
Song 12: "Intro"
Song 13: "OXXXYMIRON"
Song 14: "Russky Cockney"
Song 15: "Shade 45 Freestyle (Идея)"
Song 16: "Street Freestyle battle"
Song 17: "Ultima Thule"
Song 18: "Unreleased Track"
Song 19: "Unreleased Track 2"
Song 20: "Unreleased Track 3"
Song 21: "Unreleased Track 4"
Song 22: "XXX SHOP"
Song 23: "Башня из слоновой кости (Ivory Tower)"
Song 24: "Биполярочка (Bipolarochka)"
Song 25: "Больше Бена (Bigga Than Ben)"
Song 26: "В бульбуляторе (In the Bong)"
Song 27: "В говне (In Shit)"
Song 28: "В долгий путь (1 раунд 17ib) (On a Long Journey)"
Song 29: "Ветер перемен (2 раунд 17ib) (The Wind of Change)"
Song 30: "Вечный жид (Everlasting Jew)"
Song 31: "Витязи словоблудия (Уховертка) (The Knights

In [0]:
# saving the lyrics of 50 songs of Oxxxymiron as .txt file
artist.save_lyrics(extension='txt')

In [0]:
lyricss = open("Lyrics_Oxxxymiron.txt","r")

In [0]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [0]:
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in lyricss]

In [0]:
print(len(tokenized_text))

3201


In [0]:
# Let's define a function that will generate rap
def generate_rap (model, num_lines = 5, num_words_in_line = 10, random_seed = 135):
  for i in range(num_lines):
    content = []
    for token in model.generate(num_words_in_line, random_seed=random_seed+i):
      if token == '<s>':
        continue
      if token == '</s>':
        break
      content.append(token)
    content = TreebankWordDetokenizer().detokenize(content)
    if bool(content):
      i = 0
      while not content[i].isalpha():
          i += 1
          if i > len(content)-1:
            break
      print(content[i:].capitalize())

In [0]:
train, test = train_test_split(tokenized_text, test_size=0.2, random_state=135)

##**Unigram** **models**

In [0]:
#I will use MLE, Laplace and KNI models, and compare them with the perplexity
model = MLE(1)
train_data, padded_sents = padded_everygram_pipeline(1, train)
model.fit(train_data, padded_sents)

model1 = Laplace(1)
train_data1, padded_sents1 = padded_everygram_pipeline(1, train)
model1.fit(train_data1, padded_sents1)

model2 = KneserNeyInterpolated(1)
train_data2, padded_sents2 = padded_everygram_pipeline(1, train)
model2.fit(train_data2, padded_sents2)

In [0]:
generate_rap(model, random_seed = 135)

Они неандертальцем на камень социальными, в надо bitch ни
Мы ложь и реальными звания? ни, — победою
За — бардак господи соты укладывает куда я сюр
The и пламя « тления бреду``и говорят пока
Пиздец машина у чёрный о простых методик лифт ты


In [0]:
generate_rap(model1, random_seed = 135)

Пара но налит кричишь стань: военкоматы нба loves нужен
На мессии из рэп и double нутом, — полна
Зови қалай будь джойстике стану успел магомета это так
Алёша как подошвой белый только в hah их дела похоже
Под мне увижу чтобы оставил районам мои меня у


In [0]:
generate_rap(model2, random_seed = 135)

Поняли пирсы оставляя мониторов сотру бери дружный отчизне вам плюс
Опа недотёпа лето сакмарову курсиву бродяги по left яиц продаёт
Still красивый этанол городит изолента сотни тусклой наследил чендлер стенокардийным
Всеми любых приму вытащи таблице далёкую бывал ломбарде ибо прохода
S придумал неустанно требующая харуки покидаю рифмоваться новой невнятные точно


Generated with unigram models text looks more like the a set of words. Let calculate the perplexity for unigrams. 

In [0]:
test_data1,_ = padded_everygram_pipeline(1, test)

In [0]:
test_gen1 = []

for line in test_data1:
  test_gen1.extend(line)

In [0]:
print('MLE unigram model perplexity: {}'.format(model.perplexity(test_gen1)))
print('Laplace unigram model perplexity: {}'.format(model1.perplexity(test_gen1)))
print('KNI unigram model perplexity: {}'.format(model2.perplexity(test_gen1)))

MLE unigram model perplexity: inf
Laplace unigram model perplexity: 1433.9464746295664
KNI unigram model perplexity: 6623.999999995705


MLE model has the infinite perplexity, and it is true for all MLE models in this data set. The possible explanation that I found is that the is infinite perplexity if  words in test data are out of vocab of train data. Among two other models, Laplace shows better results.



##**Bigram models**

In [0]:
model3 = MLE(2)
train_data3, padded_sents3 = padded_everygram_pipeline(2, train)
model3.fit(train_data3, padded_sents3)

model4 = KneserNeyInterpolated(2)
train_data4, padded_sents4 = padded_everygram_pipeline(2, train)
model4.fit(train_data4, padded_sents4)

model5 = Laplace(2)
train_data5, padded_sents5 = padded_everygram_pipeline(2, train)
model5.fit(train_data5, padded_sents5)

In [0]:
generate_rap(model3, random_seed = 135)

На плите кипятку
Их подлости
Когда я дон жуан



In [0]:
generate_rap(model4, random_seed = 135)

Понял все нигеры напоминают пидоров
Опа, кто с начала движа гроши, я развиваюсь
Start this is back, у тебя мёртвым не способен
Вселяют ужас тебя? перелет короткий был зачат дабстеп?
S never slowing my whole life on the middle


In [0]:
generate_rap(model5, random_seed = 135)

Нба!
Концепт: « погоди, буду резать, ясный хрен
Короче



Using bigram models helps us to get the more connected resluts. The lines and phrases are more coherent. Let's have a look at the perplexity

In [0]:
test_data2,_ = padded_everygram_pipeline(2, test)


In [0]:
test_gen2 = []

for line in test_data2:
  test_gen2.extend(line)


In [0]:
print('MLE bigram model perplexity: {}'.format(model3.perplexity(test_gen2)))
print('Laplace bigram model perplexity: {}'.format(model5.perplexity(test_gen2)))
print('KNI bigram model perplexity: {}'.format(model4.perplexity(test_gen2)))

MLE bigram model perplexity: inf
Laplace bigram model perplexity: 1312.3297697086282
KNI bigram model perplexity: 3348.3528586536167


The bigram Laplace model shows better results, than in unigram model (1312 vs 1433). KNI has a very large value of perplexity.

##**Trigram models**

In [0]:
model6 = MLE(3)
train_data6, padded_sents6 = padded_everygram_pipeline(3, train)
model6.fit(train_data6, padded_sents6)

model7 = Laplace(3)
train_data7, padded_sents7 = padded_everygram_pipeline(3, train)
model7.fit(train_data7, padded_sents7)

model8 = KneserNeyInterpolated(3)
train_data8, padded_sents8 = padded_everygram_pipeline(3, train)
model8.fit(train_data8, padded_sents8)

In [0]:
generate_rap(model6, random_seed = 135)

Концерты
Дам слово, что он властный социопат? »
Когда что-то роняют



In [0]:
generate_rap(model7, random_seed = 135)

Мандельштамом
Ждём кислорода
Короче



In [0]:
generate_rap(model8, random_seed = 135)

Понял все трюки на битах
Опа, а с виду! »
Start this
Вселяют ужас и страх, увеселяют фристайлом, став рэп-автоматами
S never been


In terms of meaning, bigrams and trigrams are little different. Trigrams produce shorter phrases that are still unconnected.

In [0]:
test_data3,_ = padded_everygram_pipeline(3, test)

In [0]:
test_gen3 = []

for line in test_data3:
  test_gen3.extend(line)

In [0]:
print('MLE trigram model perplexity: {}'.format(model6.perplexity(test_gen3)))
print('Laplace trigram model perplexity: {}'.format(model7.perplexity(test_gen3)))
print('KNI trigram model perplexity: {}'.format(model8.perplexity(test_gen3)))

MLE trigram model perplexity: inf
Laplace trigram model perplexity: 935.1007539948323
KNI trigram model perplexity: 1501.3966443325182


Nevertheless, the trigram models show the best results for perplexity. Laplace function is still the leader. However, KNI function shows a significantly better results in trigrams than in bigrams.

The next step is to get more data and compare the results.

#**Getting more data**

I've decided to increase the sample by adding 3 new musicians - Skriptonite, Face and Slava KPSS. I do not really know whether they are similar to Oxxymiron in style and sense, because I am not a rap expert

In [0]:
genius.remove_section_headers = True
genius.skip_non_songs = False 
genius.excluded_terms = ["(Remix)", "(Live)"]

In [0]:
#maximum number of songs for each raper is 50, as it was for Oxxximiron
Scr = genius.search_artist("Скриптонит", max_songs=50, sort="title")
Face = genius.search_artist("FACE", max_songs=50, sort="title")
Slava = genius.search_artist("Слава КПСС", max_songs=50, sort="title")

Searching for songs by Скриптонит...

Changing artist name to 'Скриптонит (Scriptonite)'
Song 1: "1000"
Song 2: "100 поцелуев (100 Kisses)"
Song 3: "104 (Skit)"
Song 4: "21"
Song 5: "4 My Town"
Song 6: "5 здесь 5 там (5 zdes, 5 tam)"
Song 7: "7182"
Song 8: "Bad Vision"
Song 9: "Blunts"
Song 10: "Damn"
Song 11: "Go dilla"
Song 12: "High Volume"
Song 13: "Na Na Na"
Song 14: "No Hook"
Song 15: "No Hook #2"
Song 16: "No Hook #3 [Траур]"
Song 17: "Outro"
Song 18: "Player In C (Cover)"
Song 19: "PVL Is Back"
Song 20: "Snippet"
Song 21: "Supa Dupa"
Song 22: "VBVVCTND"
Song 23: "Work"
Song 24: "Ага, ну (Yeah, well)"
Song 25: "Больше (More)"
Song 26: "Братик (Bro)"
Song 27: "Братик 2 (Bro 2)"
Song 28: "Бумажки (Paper)"
Song 29: "Бунт (Bunt)"
Song 30: "Вечеринка (Party)"
Song 31: "Витамин (Vitamin)"
Song 32: "Внатуре (For Real)"
Song 33: "Вниз (Down)"
Song 34: "Вчера ночью (Last Night)"
Song 35: "Выпить тебя до дна (Drink you to the bottom)"
Song 36: "Выходные (Vyhodnye)"
Song 37: "Где твоя любо

In [0]:
Scr.save_lyrics(extension='txt')
Face.save_lyrics(extension='txt')
Slava.save_lyrics(extension='txt')

Wrote `Lyrics_СкриптонитScriptonite.txt`
Wrote `Lyrics_FACE.txt`
Wrote `Lyrics_СлаваКПССSlavaKPSS.txt`


In [0]:
Slava_lyr= open("Lyrics_СлаваКПССSlavaKPSS.txt","r")
Scr_lyr = open("Lyrics_СлаваКПССSlavaKPSS.txt","r")
Face_lyr = open("Lyrics_FACE.txt","r")

In [0]:
Slava_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Slava_lyr]
Scr_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Scr_lyr]
Face_tokenized = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in Face_lyr]

In [33]:
lyrics = [] #setting one list for all 4 musicians
lyrics.extend(tokenized_text)
lyrics.extend(Slava_tokenized)
lyrics.extend(Scr_tokenized)
lyrics.extend(Face_tokenized)
print(len(lyrics))

10699


In [0]:
train, test = train_test_split(lyrics, test_size=0.25, random_state=135)

## **Unigram models**

In [0]:
model = MLE(1)
train_data, padded_sents = padded_everygram_pipeline(1, train)
model.fit(train_data, padded_sents)

model1 = Laplace(1)
train_data1, padded_sents1 = padded_everygram_pipeline(1, train)
model1.fit(train_data1, padded_sents1)

model2 = KneserNeyInterpolated(1)
train_data2, padded_sents2 = padded_everygram_pipeline(1, train)
model2.fit(train_data2, padded_sents2)

In [0]:
generate_rap(model, random_seed = 135)

Она не на клешни стиль, вам навальный? нету
Мы людей и русский знает . ни, — поводу
За – бесплатный грустная стерёг фанатки ледовым я тараканов
Tonight иди плазмой а тот будто: и гордо помесь
Пиздатый меня уже шмотье нынче пьедестал мне любим увидел


In [0]:
generate_rap(model1, random_seed = 135)

Пара но налит кричишь стань: военкоматы нба loves нужен
На мессии из рэп и double нутом, — полна
Зови қалай будь джойстике стану успел магомета это так
Алёша как подошвой белый только в hah их дела похоже
Под мне увижу чтобы оставил районам мои меня у


In [40]:
generate_rap(model2, random_seed = 135)

Поставить плывущих отыграл мс сплифом болезней ебём парку вес подкрепление
Отдали неприятны лечить свалил кыргызстан бэттлиться поднял original ядерный проснешься
Yeezy кроватку эрнесто грязная иремель сплетению убогий нацист чендлер странной
Вынимать майонез продолжаю гликодин творец дерьма ввысь-ввысь лучший изнутри путь
Но проблемах ногами туз хейтер помог ротацию обернулся нему трусы-неделька


In [0]:
test_data1,_ = padded_everygram_pipeline(1, test)


In [0]:
test_gen1 = []

for line in test_data1:
  test_gen1.extend(line)


In [0]:
print('MLE unigram model perplexity: {}'.format(model.perplexity(test_gen1)))
print('Laplace unigram model perplexity: {}'.format(model1.perplexity(test_gen1)))
print('KNI unigram model perplexity: {}'.format(model2.perplexity(test_gen1)))

MLE unigram model perplexity: inf
Laplace unigram model perplexity: 1504.3992866843134
KNI unigram model perplexity: 11983.99999998177


Compared to unigram models built only on Oxxxymiron lyrics, these models show higher results, so it means that using more data gets our models a liitle bit worse for understanding.

##**Bigram models**

In [0]:
model3 = MLE(2)
train_data3, padded_sents3 = padded_everygram_pipeline(2, train)
model3.fit(train_data3, padded_sents3)

model4 = KneserNeyInterpolated(2)
train_data4, padded_sents4 = padded_everygram_pipeline(2, train)
model4.fit(train_data4, padded_sents4)

model5 = Laplace(2)
train_data5, padded_sents5 = padded_everygram_pipeline(2, train)
model5.fit(train_data5, padded_sents5)

In [0]:
generate_rap(model3, random_seed = 135)

На площадке
Интелекта снижены
Копоть



In [37]:
generate_rap(model4, random_seed = 135)

Поставит на отпуск мы сами брахманы
Отдай мне не с небес и похуй) — серп
Ya, я высоко, фристайл за пазухой, чем
Вынесено
Но ты не теряю счёт


In [38]:
generate_rap(model5, random_seed = 135)

Нба!
Концепт: « погоди, буду резать, ясный хрен
Короче



In [0]:
test_data2,_ = padded_everygram_pipeline(2, test)


In [0]:
test_gen2 = []

for line in test_data2:
  test_gen2.extend(line)

In [0]:
print('MLE bigram model perplexity: {}'.format(model3.perplexity(test_gen2)))
print('Laplace bigram model perplexity: {}'.format(model5.perplexity(test_gen2)))
print('KNI bigram model perplexity: {}'.format(model4.perplexity(test_gen2)))

MLE bigram model perplexity: inf
Laplace bigram model perplexity: 1272.2152578934601
KNI bigram model perplexity: 2040.6538711956744


These results are better, than for bigram models for a smaller dataset. The best model is Laplace one.

##**Trigram models**

In [0]:
model6 = MLE(3)
train_data6, padded_sents6 = padded_everygram_pipeline(3, train)
model6.fit(train_data6, padded_sents6)

model7 = Laplace(3)
train_data7, padded_sents7 = padded_everygram_pipeline(3, train)
model7.fit(train_data7, padded_sents7)

model8 = KneserNeyInterpolated(3)
train_data8, padded_sents8 = padded_everygram_pipeline(3, train)
model8.fit(train_data8, padded_sents8)

In [0]:
generate_rap(model6, random_seed = 15)

Китайцев
Гордо течет (гордо течет (гордо течет [?
Копоть



In [0]:
generate_rap(model7, random_seed = 15)

Ы) да
Каждый квартал check, словно оборотень
Выкини коленце, в стойбище прет жеребёнок
Метро как анаконда и жёлтая дорожка


In [0]:
generate_rap(model8, random_seed = 135)

Поставит на репит музыку сфер
Отдай мне злато
Ya, ya, ya
Вынесено
Но ты ведь так хотел


In [0]:
test_data3,_ = padded_everygram_pipeline(3, test)


In [0]:
test_gen3 = []

for line in test_data3:
  test_gen3.extend(line)

In [0]:
print('MLE trigram model perplexity: {}'.format(model6.perplexity(test_gen3)))
print('Laplace trigram model perplexity: {}'.format(model7.perplexity(test_gen3)))
print('KNI trigram model perplexity: {}'.format(model8.perplexity(test_gen3)))

MLE trigram model perplexity: inf
Laplace trigram model perplexity: 875.6413284752927
KNI trigram model perplexity: 585.7741052536325


The trigram models for a big dataset shave shown the best results for the whole work. Surprisingly, the KNI perplexity is several time lower than for smaller dataset. So, we can conclude that the KNI trigram model with the bigger dataset is the best choice for this case. From the point of view of sense, the result of this model is also could be considered as something more or less reasonable.